# CSV De-identification using a Local LLM with Ollama

This notebook processes a user-specified CSV file to de-identify text in one or more columns. It uses a locally running LLM, accessed via the Ollama API, to identify and replace Protected Health Information (PHI) with category placeholders (e.g., `[PERSON]`, `[DATE]`).

### How to Use:
1.  **Fill out the Configuration section below.** You must provide the path to your input CSV and a list of the column names you want to clean.
2.  **Run all cells.** The script will process your file in batches and create new, de-identified CSV files in the same directory.

### Key Features:

- **Live Progress Tracking**: See which row is being processed in real-time for each batch.
- **User-Friendly Setup**: No need to create sample data; just point the script to your file.
- **Multi-Column Processing**: De-identify one or more text columns in the same run.
- **Flexible Output**: Option to either replace the PHI column (default) or add a new de-identified column.
- **Large File Support (Batching)**: Automatically splits large CSVs into smaller, numbered batches.
- **Resumability**: If the script is stopped, it can be rerun and will automatically skip any completed batches.
- **Error Handling**: Rows with errors are retried up to 4 times before being marked as "unable to deidentify".

### Prerequisites:

1.  **Ollama is installed and running**: You must have Ollama installed on your system.
2.  **Gemma model is available**: You need to have pulled a model. You can do this by running `ollama pull gemma3:4b` in your terminal.
3.  **Python Libraries**: Ensure you have `pandas` and `requests` installed (`pip install pandas requests`).

In [1]:
import pandas as pd
import requests
import json
import math
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

## 1. Configuration (User Input Required)
**Please fill in the variables in the cell below before running the notebook.**

In [2]:
# --- REQUIRED: Specify the path to your CSV file ---
INPUT_CSV_PATH = "/Users/dli989/Documents/RECOVER-local/liebovitz-llm_deidentified_notes.csv" # Example: "C:/Users/YourUser/Desktop/my_notes.csv"

# --- REQUIRED: List the names of the columns you want to de-identify ---
COLUMNS_TO_CLEAN = ["note_text"] # Example: ["clinical_notes", "another_column_with_phi"]

# --- OPTIONAL: Advanced Settings ---
REPLACE_ORIGINAL_COLUMN = True      # True: Replaces original column(s). False: Adds new de-identified column(s).
OUTPUT_PREFIX = "deidentified_output_post_LLM" # The prefix for the output files (e.g., deidentified_output_part_1.csv)
MAX_ROWS_PER_BATCH = 300         # The max number of rows to process in a single batch file.
MAX_WORKERS = 10                   # Number of parallel threads for API calls (adjust based on your system)
MAX_RETRIES = 4                    # Number of times to retry processing a row before marking it as "unable to deidentify"

# --- OPTIONAL: Ollama Settings ---
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "gemma3:4b"             # Or any other model you have available
MAX_CHUNK_SIZE = 5000               # Max characters for a single note before it's split for processing.

## 2. Core Functions

These functions handle the communication with the Ollama API and the logic for chunking and reassembling the text for a single note.

In [3]:
def call_ollama_api(text_chunk, row_index=None, retry_count=0):
    """
    Sends a text chunk to the Ollama API for de-identification.
    Includes retry logic to attempt processing up to MAX_RETRIES times.
    """
    if retry_count >= MAX_RETRIES:
        return "[UNABLE TO DEIDENTIFY: Maximum retry attempts reached]"
    
    prompt = f"""You are a highly accurate de-identification tool. Your only task is to find and replace all Protected Health Information (PHI) in the following text with a category label in brackets. 
    Replace names with [PERSON], dates with [DATE], locations with [LOCATION], phone numbers with [PHONE], email addresses with [EMAIL], identification numbers (like SSN, MRN) with [ID_NUMBER].
    ABSOLUTELY DO NOT include any commentary, greetings, or explanations in your response. Only return the modified text. Be careful to recognize if first name and last name are together, e.g,, "jennifersmith" which should be replaced with [PERSON].
    
    Here is the text:
    ---
    {text_chunk}
    ---"""
    
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, json=payload, timeout=120)
        response.raise_for_status()
        response_data = response.json()
        return response_data.get('response', '').strip()
    except requests.exceptions.RequestException as e:
        # The leading \n is removed to avoid breaking the progress bar line.
        retry_count += 1
        if row_index is not None:
            print(f"\nError processing row {row_index}: {e} (Attempt {retry_count}/{MAX_RETRIES})")
        
        # Wait briefly before retrying to avoid overwhelming the API
        if retry_count < MAX_RETRIES:
            time.sleep(2)  # Short delay before retry
            return call_ollama_api(text_chunk, row_index, retry_count)
        else:
            return "[UNABLE TO DEIDENTIFY: Maximum retry attempts reached]"

def deidentify_text(full_text, row_index=None):
    """
    Manages the de-identification of a full text string, handling chunking if necessary.
    """
    if not isinstance(full_text, str) or not full_text.strip():
        return ""
    
    if len(full_text) <= MAX_CHUNK_SIZE:
        return call_ollama_api(full_text, row_index)
    
    chunks = [full_text[i:i + MAX_CHUNK_SIZE] for i in range(0, len(full_text), MAX_CHUNK_SIZE)]
    processed_chunks = []
    
    for i, chunk in enumerate(chunks):
        processed_chunk = call_ollama_api(chunk, row_index)
        processed_chunks.append(processed_chunk)
        
    return "".join(processed_chunks)

def process_row_parallel(args):
    """
    Helper function for parallel processing of individual rows.
    Returns tuple of (original_index, processed_text) to maintain order.
    """
    row_index, original_text = args
    processed_text = deidentify_text(original_text, row_index)
    return (row_index, processed_text)

## 3. Main Processing and Batching Logic

This section reads the CSV, splits it into batches, and processes each specified column row by row to provide a live progress update.

In [4]:
def process_large_csv(input_path, output_prefix, columns_to_clean):
    """
    Reads a potentially large CSV, splits it into batches, de-identifies the 
    specified columns, and saves new numbered CSV files for each batch.
    """
    if not os.path.exists(input_path) or input_path == "/path/to/your/file.csv":
        print("ERROR: Input file not found or path not set.")
        print(f"Please update the 'INPUT_CSV_PATH' variable in the Configuration section.")
        return
    
    # Get the directory of the input file to save output files in the same location
    output_dir = os.path.dirname(os.path.abspath(input_path))
    
    try:
        print(f"Reading and preparing CSV from '{input_path}'...")
        df_iterator = pd.read_csv(input_path, chunksize=MAX_ROWS_PER_BATCH, on_bad_lines='warn')
        with open(input_path, 'r', encoding='utf-8', errors='ignore') as f:
            total_rows = sum(1 for row in f) - 1 # -1 for header
        num_batches = math.ceil(total_rows / MAX_ROWS_PER_BATCH)

    except Exception as e:
        print(f"Error reading input file: {e}")
        return

    print(f"Total rows: {total_rows}. This will be processed in {num_batches} batch(es).")
    print(f"Output files will be saved in: {output_dir}")
    
    # Track processed batches for summary
    processed_batches = []
    skipped_batches = []

    for i, batch_df in enumerate(df_iterator):
        batch_num = i + 1
        output_path = os.path.join(output_dir, f"{output_prefix}_part_{batch_num}.csv")
        
        if os.path.exists(output_path):
            print(f"\nOutput file '{output_path}' already exists. Skipping Batch {batch_num}.")
            skipped_batches.append(batch_num)
            continue

        print(f"\n--- Processing Batch {batch_num}/{num_batches} ---")
        
        for column_name in columns_to_clean:
            if column_name not in batch_df.columns:
                print(f"  - WARNING: Column '{column_name}' not found in this batch. Skipping.")
                continue
            
            print(f"  - De-identifying column: '{column_name}' using {MAX_WORKERS} parallel workers")
            
            total_in_batch = len(batch_df)
            
            # Prepare data for parallel processing: (row_index, text_content)
            row_data = [(idx, row[column_name]) for idx, row in batch_df.iterrows()]
            
            # Initialize results dictionary to maintain order
            processed_results = {}
            completed_count = 0
            
            # Use ThreadPoolExecutor for parallel processing
            with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                # Submit all tasks
                future_to_index = {executor.submit(process_row_parallel, data): data[0] for data in row_data}
                
                # Process completed tasks as they finish
                for future in as_completed(future_to_index):
                    original_index, processed_text = future.result()
                    processed_results[original_index] = processed_text
                    completed_count += 1
                    
                    # Print progress
                    print(f"\r    - Row {completed_count}/{total_in_batch}", end="")
            
            print() # Newline after the progress bar for a column is complete
            
            # Reconstruct the processed data in the original order
            processed_data = [processed_results[idx] for idx, _ in batch_df.iterrows()]
            
            # Update the dataframe with processed data
            if REPLACE_ORIGINAL_COLUMN:
                batch_df[column_name] = processed_data
            else:
                batch_df[f"{column_name}_deidentified"] = processed_data
        
        # Save the processed batch to CSV
        try:
            batch_df.to_csv(output_path, index=False)
            print(f"  - Saved batch to: '{output_path}'")
            processed_batches.append(batch_num)
        except Exception as e:
            print(f"  - ERROR: Could not save batch {batch_num}: {e}")
    
    print(f"\n--- Processing Complete ---")
    print(f"All batches have been processed and saved with prefix '{output_prefix}'.")
    
    # Print summary of processed and skipped batches
    if processed_batches:
        print(f"\nProcessed batches: {', '.join(map(str, processed_batches))}")
    if skipped_batches:
        print(f"Skipped batches (already existed): {', '.join(map(str, skipped_batches))}")
    
    # Print the location of the output files
    print(f"\nOutput files are located in: {output_dir}")
    print(f"File naming pattern: {output_prefix}_part_X.csv where X is the batch number")
    
    # List the output files that exist
    existing_output_files = [f for f in os.listdir(output_dir) if f.startswith(output_prefix) and f.endswith('.csv')]
    if existing_output_files:
        print(f"\nFound {len(existing_output_files)} output files:")
        for file in sorted(existing_output_files):
            file_path = os.path.join(output_dir, file)
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convert to MB
            print(f"  - {file} ({file_size:.2f} MB)")
    else:
        print(f"\nNo output files found with prefix '{output_prefix}' in {output_dir}")

## 4. Run the Process

Execute the main function. This will start the process using the file and columns you specified in the Configuration section. If you run this cell a second time, it will find the generated output files and skip the processing.

In [ ]:
# This cell runs the main function with the settings you provided at the top.
process_large_csv(
    input_path=INPUT_CSV_PATH,
    output_prefix=OUTPUT_PREFIX, 
    columns_to_clean=COLUMNS_TO_CLEAN
)

Reading and preparing CSV from '/Users/dli989/Documents/RECOVER-local/liebovitz-llm_deidentified_notes.csv'...
Total rows: 604482. This will be processed in 2015 batch(es).
Output files will be saved in: /Users/dli989/Documents/RECOVER-local

Output file '/Users/dli989/Documents/RECOVER-local/deidentified_output_post_LLM_part_1.csv' already exists. Skipping Batch 1.

Output file '/Users/dli989/Documents/RECOVER-local/deidentified_output_post_LLM_part_2.csv' already exists. Skipping Batch 2.

Output file '/Users/dli989/Documents/RECOVER-local/deidentified_output_post_LLM_part_3.csv' already exists. Skipping Batch 3.

Output file '/Users/dli989/Documents/RECOVER-local/deidentified_output_post_LLM_part_4.csv' already exists. Skipping Batch 4.

Output file '/Users/dli989/Documents/RECOVER-local/deidentified_output_post_LLM_part_5.csv' already exists. Skipping Batch 5.

Output file '/Users/dli989/Documents/RECOVER-local/deidentified_output_post_LLM_part_6.csv' already exists. Skipping Batch